# Model usage example

In [1]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

from spark.config import views
from spark.create_session import create_session

from IPython.display import display


Create Spark session and load the data. Files should be stored in the `data/{version}` directory under project root.

In [2]:
VIEWS = views("v4")
spark = create_session()

for view, file in VIEWS.items():
    df = spark.read.json(file)
    df.createOrReplaceTempView(view)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/05 17:27:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Query for users, tracks and sessions. Sessions query is performed here only to preview the data, a more complex sessions query will be performed later on.

In [151]:
sessions_full = spark.sql(f"SELECT * FROM sessions").toPandas()
sessions = spark.sql(f"SELECT DISTINCT user_id, track_id FROM sessions WHERE event_type like 'like' order by user_id, track_id").toPandas()
_tracks = spark.sql(
    f"SELECT DISTINCT id, id_artist, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, popularity, EXTRACT(year from `release_date`) as release_year, speechiness, tempo, valence FROM tracks ").toPandas()
users = spark.sql(f"SELECT user_id FROM users").toPandas()

In [152]:
display(sessions_full)
display(sessions)
display(_tracks)
display(users)

,event_type,session_id,timestamp,track_id,user_id
0,play,124,2023-05-30T23:37:17,0qbV4e18lWrTTxlswVoLbI,101
1,advertisment,124,2023-05-30T23:44:37.941000,None,101
2,like,124,2023-05-30T23:44:37.941000,0qbV4e18lWrTTxlswVoLbI,101
3,skip,124,2023-05-30T23:44:46.212000,0qbV4e18lWrTTxlswVoLbI,101
4,play,124,2023-05-30T23:44:51.212000,3pyTksNccLM1jRvzQ4zTke,101
...,...,...,...,...,...
12340426,play,878516,2022-12-27T11:20:54.236000,2nxzEMUmIhSJeGBaXgh30x,5100
12340427,play,878516,2022-12-27T11:26:11.409000,19K3lUMJmOdeuOBTrbLm19,5100
12340428,play,878516,2022-12-27T11:30:27.182000,1PhLYngBKbeDtdmDzCg3Pb,5100
12340429,like,878516,2022-12-27T11:32:22.571000,1PhLYngBKbeDtdmDzCg3Pb,5100


,user_id,track_id
0,101,01q4ccXbvPlCwZ1fPiFaeM
1,101,02ePjHjIiszSYqeLykvpTN
2,101,02ppMPbg1OtEdHgoPqoqju
3,101,03EnOL1O9EKi9CFNmPyrCm
4,101,0534jmQ0dYChW5MSzYXNVr
...,...,...
2568073,5100,7xD7MvjAdZkx1YICschIuI
2568074,5100,7xa3dJQMzBVzsrZ81tNcHP
2568075,5100,7yYvvOB7CuzdVldb6zOk1m
2568076,5100,7ygpwy2qP3NbrxVkHvUhXY


,id,id_artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
0,4y8icjzu6fZP503Mg31Tpn,5V0MlUE1Bft0mbLlND7FJz,0.87800,0.184,224227,0.292,0.000258,7,0.3460,-12.246,55,1956,0.0312,75.630,0.1430
1,4hHbeIIKO5Y5uLyIEbY9Gn,1Mxqyy3pSjf8kZZL4QVxS0,0.84500,0.574,199093,0.338,0.000000,6,0.1650,-11.376,69,1958,0.0420,67.008,0.4930
2,0KSHmjK7OFtGocvbo7NZNO,6kACVPfCOnqzgfEF5ryl0x,0.80300,0.754,107280,0.484,0.000009,10,0.0989,-9.196,52,1959,0.0609,82.761,0.8010
3,30V02AmDYMRvrHE4L8cZAo,22bE4uQ6baNwSHPVcDxLCe,0.17800,0.578,133320,0.429,0.000060,2,0.1710,-9.601,57,1965,0.0268,108.738,0.0889
4,2CQRYn5cTD2B9a1ONjhTN2,3oDbviiivRWhXwIE8hxkVV,0.72500,0.396,145800,0.686,0.000001,5,0.1700,-5.607,57,1966,0.0341,124.318,0.7170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22407,3atrFJhUzDm5xiutqfEPkP,74XFHRwlV6OrjEM0A2NCMF,0.00576,0.489,218333,0.611,0.000008,6,0.0917,-5.675,52,2007,0.0337,140.956,0.1860
22408,3wYzd4dzWknPdgwNaPGQvg,7JthQ6zwNzfxRfIEjp6wUs,0.06960,0.607,191474,0.605,0.000000,3,0.4540,-6.050,65,2021,0.1590,83.913,0.6650
22409,6UIcN1tiiGdd7oMMzNvyaP,7lbSsjYACZHn1MSDXPxNF2,0.06010,0.479,297502,0.478,0.000007,5,0.0813,-8.884,53,2020,0.1100,100.040,0.1170
22410,62i2LI6iDmAHbE9H9wa99z,7k73EtZwoPs516ZxE72KsO,0.38500,0.470,216970,0.738,0.000621,8,0.0656,-4.641,52,2017,0.0398,97.973,0.4880


,user_id
0,101
1,102
2,103
3,104
4,105
...,...
4995,5096
4996,5097
4997,5098
4998,5099


Feature engineering goes here. For now, standardize all numeric features in `tracks` ($\mu=0$, $\sigma=1$).

In [160]:
from scipy import stats

# normalize track features
tracks = pd.concat([_tracks[['id', 'id_artist']], _tracks.drop(['id', 'id_artist'], axis=1).apply(stats.zscore)], axis=1)
display(tracks)

,id,id_artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
0,4y8icjzu6fZP503Mg31Tpn,5V0MlUE1Bft0mbLlND7FJz,2.210183,-2.622956,-0.081994,-1.698778,-0.261491,0.483649,0.910200,-1.350924,-0.792191,-3.602614,-0.579005,-1.562809,-1.533923
1,4hHbeIIKO5Y5uLyIEbY9Gn,1Mxqyy3pSjf8kZZL4QVxS0,2.090663,-0.158580,-0.430623,-1.479284,-0.263179,0.202650,-0.167568,-1.118180,0.947358,-3.458942,-0.458085,-1.853949,-0.102263
2,0KSHmjK7OFtGocvbo7NZNO,6kACVPfCOnqzgfEF5ryl0x,1.938547,0.978824,-1.704140,-0.782628,-0.263120,1.326645,-0.561162,-0.534983,-1.164951,-3.387106,-0.246475,-1.322015,1.157598
3,30V02AmDYMRvrHE4L8cZAo,22bE4uQ6baNwSHPVcDxLCe,-0.325086,-0.133305,-1.342945,-1.045067,-0.262786,-0.921346,-0.131841,-0.643329,-0.543684,-2.956090,-0.628269,-0.444846,-1.755216
4,2CQRYn5cTD2B9a1ONjhTN2,3oDbviiivRWhXwIE8hxkVV,1.656046,-1.283347,-1.169838,0.181238,-0.263172,-0.078349,-0.137796,0.425153,-0.543684,-2.884254,-0.546536,0.081246,0.814000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22407,3atrFJhUzDm5xiutqfEPkP,74XFHRwlV6OrjEM0A2NCMF,-0.948906,-0.695688,-0.163749,-0.176633,-0.263129,0.202650,-0.604035,0.406961,-1.164951,0.061025,-0.551015,0.643063,-1.358033
22408,3wYzd4dzWknPdgwNaPGQvg,7JthQ6zwNzfxRfIEjp6wUs,-0.717690,0.049944,-0.536304,-0.205263,-0.263179,-0.640347,1.553288,0.306641,0.450344,1.066730,0.851883,-1.283116,0.601296
22409,6UIcN1tiiGdd7oMMzNvyaP,7lbSsjYACZHn1MSDXPxNF2,-0.752097,-0.758877,0.934387,-0.811258,-0.263132,-0.078349,-0.665962,-0.451516,-1.040697,0.994894,0.303264,-0.738553,-1.640275
22410,62i2LI6iDmAHbE9H9wa99z,7k73EtZwoPs516ZxE72KsO,0.424630,-0.815747,-0.182655,0.429362,-0.259116,0.764647,-0.759448,0.683579,-1.164951,0.779385,-0.482717,-0.808350,-0.122715


Query for sessions:
- Group user-track interactions (this can be used to create event weights later on)
- Exclude tracks from `sessions.jsonl` not present in `tracks.jsonl`
- Filter event type

In [161]:
d = spark.sql(
    """
    SELECT s.user_id, s.track_id, s.weight, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, popularity, EXTRACT(year from `release_date`) as release_year, speechiness, tempo, valence
    FROM (
        select user_id, track_id, sum(event_weight) as weight
        from (
            SELECT user_id, track_id, 1 as event_weight
            FROM sessions
            WHERE event_type like 'like'
            ) 
        group by user_id, track_id
    ) s
    inner join tracks t on s.track_id = t.id
    order by s.user_id, t.id
    """).toPandas()
d

,user_id,track_id,weight,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
0,101,01q4ccXbvPlCwZ1fPiFaeM,1,0.126000,0.680,122760,0.626,0.000001,7,0.0995,-8.519,57,1987,0.0255,104.333,0.960
1,101,02ePjHjIiszSYqeLykvpTN,2,0.159000,0.487,220920,0.909,0.000001,9,0.3030,-3.484,54,1995,0.0463,125.418,0.510
2,101,02ppMPbg1OtEdHgoPqoqju,1,0.511000,0.523,290213,0.656,0.160000,4,0.0679,-7.441,63,2007,0.0262,104.271,0.214
3,101,03EnOL1O9EKi9CFNmPyrCm,1,0.497000,0.555,355640,0.460,0.000000,9,0.1030,-7.032,69,2005,0.0277,67.027,0.146
4,101,0534jmQ0dYChW5MSzYXNVr,1,0.019400,0.590,265960,0.711,0.000000,0,0.1220,-7.589,52,2011,0.0247,143.937,0.847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568073,5100,7xD7MvjAdZkx1YICschIuI,1,0.663000,0.544,224667,0.884,0.000007,8,0.3810,-8.166,53,1976,0.1440,127.215,0.467
2568074,5100,7xa3dJQMzBVzsrZ81tNcHP,2,0.000094,0.319,203653,0.965,0.002210,7,0.1410,-4.354,52,1992,0.0646,127.721,0.624
2568075,5100,7yYvvOB7CuzdVldb6zOk1m,1,0.001240,0.659,288973,0.846,0.000000,7,0.0415,-7.579,52,1998,0.1110,92.803,0.595
2568076,5100,7ygpwy2qP3NbrxVkHvUhXY,1,0.000509,0.375,258773,0.893,0.000000,2,0.1870,-4.097,76,1995,0.0406,174.323,0.434


Create a LightFm `Dataset`.

In [162]:
from lightfm.data import Dataset

dataset = Dataset()
# all user and track ids
dataset.fit(
    users=users['user_id'],
    items=tracks['id']
)

dataset.fit_partial(
    items=tracks['id'],
    # must follow the same order
    item_features=tracks.drop('id', axis=1)
)

num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 5000, num_items 22412.


Build the interaction matrix from user-track tuples. Weights can also be supplied.

In [163]:
(interactions, weights) = dataset.build_interactions(d[['user_id', 'track_id']].apply(tuple, axis=1))

print(repr(interactions))

<5000x22412 sparse matrix of type '<class 'numpy.int32'>'
	with 2568078 stored elements in COOrdinate format>


Split the dataset 80:20 for training and testing.

In [164]:
from lightfm.cross_validation import random_train_test_split

(train, test) = random_train_test_split(interactions)

In [165]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k, reciprocal_rank

Build an item feature matrix. This is a LightFm specific step. As we already passed feature values, we only need to supply item - feature name list tuples. LightFM uses a normalization mechanism for feature values which scales them to ensure all weights in every row sum up to 1 - we probably do not want this to happen. 

In [166]:
feature_names = tracks.drop(['id'], axis=1).columns

item_features = dataset.build_item_features(
    # track id + list of feature name pairs
    ((i, feature_names) for i in tracks['id']), 
    normalize=False)
print(repr(item_features))

<22412x22426 sparse matrix of type '<class 'numpy.float32'>'
	with 336180 stored elements in Compressed Sparse Row format>


Train the model.

In [168]:
model_h = LightFM(
    loss='warp',
    learning_rate=0.05,
    item_alpha=1e-6,
    user_alpha=1e-6,
    no_components=30,
)
model_h.fit(
    interactions=train,
    item_features=item_features,
    epochs=10,
    num_threads=12,
    verbose=True)

Epoch: 100%|██████████| 10/10 [02:14<00:00, 13.48s/it]


Evaluate metrics for single-user predictions. Train set is also included for sanity-checking (plus, to compare how well the model is able to generalize).

In [169]:
print("0/8 done...", end="")
train_auc_h = auc_score(model_h, train, 
                        item_features=item_features,
                        num_threads=12).mean()
print("\r1/8 done...", end="")
test_auc_h = auc_score(model_h, test, 
                       train_interactions=train, 
                       item_features=item_features, 
                       num_threads=12).mean()
print("\r2/8 done...", end="")

train_precision_h = precision_at_k(model_h, train, k=10,
                                   item_features=item_features, 
                                   num_threads=12).mean()
print("\r3/8 done...", end="")
test_precision_h = precision_at_k(model_h, test, k=10, 
                                  train_interactions=train,
                                  item_features=item_features, 
                                  num_threads=12).mean()
print("\r4/8 done...", end="")

train_recall_h = recall_at_k(model_h, train, k=10, 
                             item_features=item_features,
                             num_threads=12).mean()
print("\r5/8 done...", end="")
test_recall_h = recall_at_k(model_h, test, k=10, 
                            train_interactions=train,
                            item_features=item_features, 
                            num_threads=12).mean()
print("\r6/8 done...", end="")

train_reciprocal_rank_h = reciprocal_rank(model_h, train,
                                          item_features=item_features,
                                          num_threads=12).mean()
print("\r7/8 done...", end="")
test_reciprocal_rank_h = reciprocal_rank(model_h, test, 
                                         train_interactions=train,
                                         item_features=item_features,
                                         num_threads=12).mean()
print("\r8/8 done...")

print('AUC: train %.6f, test %.6f.' % (train_auc_h, test_auc_h))
print('Precision: train %.6f, test %.6f.' % (train_precision_h, test_precision_h))
print('Recall: train %.6f, test %.6f.' % (train_recall_h, test_recall_h))
print('Reciprocal rank: train %.6f, test %.6f.' % (train_reciprocal_rank_h, test_reciprocal_rank_h))

8/8 done...AUC: train 0.945076, test 0.934822.
Precision: train 0.396700, test 0.232520.
Recall: train 0.025355, test 0.097656.
Reciprocal rank: train 0.592917, test 0.341225.


Process raw scores returned by the model to get predicted track ids for both single-user and multi-user predictions.


**NOTE: these methods work a little different from recommender protocol**

In [416]:
def predict_single(user: int, number: int) -> pd.Series:
    """
    Recommends `number` of tracks to user of ID `user`.
    
    Args:
        user (int): user ID
        number (int): number of recommendations to make
        
    Returns:
         pd.Series: Predicted tracks. The series is indexed with track indices in item list passed when training model, series values are track ID's. 
    """
    predicted_scores = model_h.predict(user, np.arange(interactions.shape[1]), 
                                   item_features=item_features, 
                                   num_threads=12)
    
    return pd.concat([tracks['id'], pd.Series(predicted_scores)], axis=1).nlargest(number, 0).sort_values(by=0, ascending=False)['id']

def predict_multiple(users: list[int], number: int) -> pd.Series:
    """
    Recommend `number` of tracks to users of ID's `users`.
        
    Args:
        users (list[int]): user ID's (at least two)
        number (int): number of recommendations to make
        
    Returns:
         pd.Series: Predicted tracks. The series is indexed with track ID's (NOTE - different from `predict_single`), series values are mean ranks of items across single-user prediction for all users. The series is ordered by mean ranks.
         
    Raises:
        ValueError: If 0 or 1 user ID supplied.
    """
    if len(users) <= 1:
        raise ValueError("To make single-user predictions, use `predict_single`")
    
    predictions = [predict_single(u, tracks.size) for u in users]
    common = frozenset(predictions[0]).intersection(*(frozenset(p) for p in predictions[1:]))
    
    indexed_by_rank = (s.reset_index(drop=True).loc[s.reset_index().id.isin(common)].sort_values() for s in predictions)

    return pd.concat([pd.Series(s.index.values, index=s) for s in indexed_by_rank], axis=1).mean('columns').nsmallest(number)


predict_multiple([123, 124, 125, 126, 127, 128], 20)


id
0K1KOCeJBj3lpDYxEX9qP2    1232.666667
2iUmqdfGZcHIhS3b9E9EWq    1815.166667
2o3qA8lOCXZ8cRGhuWNT4J    1961.000000
3mTEYjm3kIm4YzJ5zxJZjs    2216.666667
0Sayb1EWTywPttwJo7zjBt    2223.500000
0rFOs9paloAvEtzwDX1Kmc    2256.500000
3xrn9i8zhNZsTtcoWgQEAd    2314.000000
1fBl642IhJOE5U319Gy2Go    2348.000000
3xrwXWG4O9uhtRyAd3MCou    2356.333333
4BUtvtGTRvJB49Hx2F7lle    2378.166667
2fweESxSLj6H8Z9HzN6LKX    2380.333333
0l4DTppOxy7NUaEcwXuOb6    2435.666667
2LlJLOK9DnYaOTr2yjqlYt    2471.000000
0pg4ptWsO9eMH6nEhrM9fD    2487.000000
4AFwnrH5atiJoAd5xS5QtR    2545.333333
5WSdMcWTKRdN1QYVJHJWxz    2555.500000
4wTChU0tU44TAMC0xcQizi    2577.833333
4PFAm88FBkY7IFtvCaGctZ    2594.000000
3UH4JIDuP83866Y43bbo4k    2617.166667
0mjAU3yKR1QnXnHtjGJqTM    2652.166667
dtype: float64

Make some test predictions for single users. They should not overlap too much.

In [417]:
n = 2000
r1 = predict_single(123, n)
r2 = predict_single(124, n)
print(r1)
print(r2)
overlap = set(r1).intersection(set(r2))
print(overlap)
print(f"{len(overlap)}/{n} ({len(overlap)*100/n:.2f}%) items overlaping")

12738    7qiZfU4dY1lWllzX7mPBI3
254      54bFM56PmE4YLRnqpW6Tha
13071    3eekarcy7kvN4yt5ZFzltW
810      7FdUvDkaE24o3FPIWTvzv2
14526    3jjujdWJ72nww5eGnfs2E7
                  ...          
14240    0dPPUcfDnvSwK38lLgzKSd
10027    2CEgGE6aESpnmtfiZwYlbV
17192    7DMPq3XndRJaj6NTINsLOz
9230     5ulZo7nk3MByXLvja3NCAc
3567     386RUes7n1uM1yfzgeUuwp
Name: id, Length: 2000, dtype: object
5401     2Rn5VhDVELkuB4BjZEVJKr
3522     06EC9TibZK2mACTZ2usK1h
19921    0IGXkVRn0uCsgn4FvIfpgB
15086    3iV11AiIEDIOVjKy3kgjld
16729    0LAcM6I7ijW4VVW0aytl1t
                  ...          
2936     3VefJ4Kkzy8zeOFK8omiOd
751      52vA3CYKZqZVdQnzRrdZt6
2858     4RU7rnJBOXCzWmsLDqUaFC
11064    7cR56TtuWIktf7jaZfyIgz
19538    5JTgqOeHWg4bxMZYMRTE4H
Name: id, Length: 2000, dtype: object
{'4cKJsZuzbNBpdxeMC9pBQG', '5H8xFHau0x3PI1CssO6EaX', '39cFQHfY6tsdMRmSbFse3X', '0aMqNFBj9KtPTD3c3tByRT', '2HklyUw3TR56nVyVJAIhw5', '3gwRWIbr5ycEVLIAXrWaw7', '1hIQPCM3oWXrpnXmgTDaKG', '0lkQOB949M2gLyut86aJ1b', '7lBPn6LpIv

Sanity check - make sure the model does not return the same items for every user.

In [418]:
r1_ = predict_single(123, 10)
r2_ = predict_single(124, 10)
assert len(set(r1_).intersection(set(r2_))) <= 1
print("Sanity check PASS")

Sanity check PASS


Evaluate the model on multi-user predictions. To compare results, measure the mean distance to center of gravity predicted items' latent representations generated by the model.

In [447]:
N = 20
REPS = 100

res_m = predict_multiple([123, 124, 125, 126, 127, 128], N)
print(res_m)


# LightFM maps provided track ID's to its own internal mapping (indices in interaction matrix). To find latent vector representations for predicted items, we first need to remap ID's to internal indices.
(_, _, item_id_mapping, _) = dataset.mapping()

def mean_dist_from_cluster_center(items_ids, model: LightFM):
    item_indices = [item_id_mapping[i] for i in items_ids]
    coords = np.take(model.get_item_representations()[1], item_indices, axis=0)
    center = np.sum(coords, axis=0) / coords.shape[0]
    return  np.average(np.apply_along_axis(lambda x: np.linalg.norm(center - x, ord=2), 1, coords))


dist_predicted = mean_dist_from_cluster_center(res_m.index, model_h)

# draw random samples from all tracks
random_items = tracks['id'].sample(N)
dist_random = sum([mean_dist_from_cluster_center(
    tracks['id'].sample(N), 
    model_h) for _ in range(REPS)]) / REPS

# draw random samples from combined users' session history
dist_sampled = sum([mean_dist_from_cluster_center(
    pd.concat(
    (d.loc[d['user_id'] == i] for i in (123, 124, 125, 126, 127, 128))
)['track_id'].sample(N), 
    model_h) for _ in range(REPS)]) / REPS

display(pd.DataFrame(
    data={
        "method": ["Predicted by model", "Sampled from history", "Entirely random"],
        "mean distance": [dist_predicted, dist_sampled, dist_random]
    }
))

id
0K1KOCeJBj3lpDYxEX9qP2    1232.666667
2iUmqdfGZcHIhS3b9E9EWq    1815.166667
2o3qA8lOCXZ8cRGhuWNT4J    1961.000000
3mTEYjm3kIm4YzJ5zxJZjs    2216.666667
0Sayb1EWTywPttwJo7zjBt    2223.500000
0rFOs9paloAvEtzwDX1Kmc    2256.500000
3xrn9i8zhNZsTtcoWgQEAd    2314.000000
1fBl642IhJOE5U319Gy2Go    2348.000000
3xrwXWG4O9uhtRyAd3MCou    2356.333333
4BUtvtGTRvJB49Hx2F7lle    2378.166667
2fweESxSLj6H8Z9HzN6LKX    2380.333333
0l4DTppOxy7NUaEcwXuOb6    2435.666667
2LlJLOK9DnYaOTr2yjqlYt    2471.000000
0pg4ptWsO9eMH6nEhrM9fD    2487.000000
4AFwnrH5atiJoAd5xS5QtR    2545.333333
5WSdMcWTKRdN1QYVJHJWxz    2555.500000
4wTChU0tU44TAMC0xcQizi    2577.833333
4PFAm88FBkY7IFtvCaGctZ    2594.000000
3UH4JIDuP83866Y43bbo4k    2617.166667
0mjAU3yKR1QnXnHtjGJqTM    2652.166667
dtype: float64


,method,mean distance
0,Predicted by model,1.101236
1,Sampled from history,1.470528
2,Entirely random,0.973284


Note that entirely randomly chosen items may have better clustering, which is expected - they are spread relatively evenly throughout the entire space. What's important is whether model predictions are more clustered than random samples from users' session history - if so, the model managed to find a "common ground" between different music tastes.

# The same example using model classes

Both models match `model.model_protocol.Recommender` protocol - it can be used for type checking.

In [454]:
from model.base_model import BaseModel
from model.fm_model import FMModel

base_model = BaseModel(
    users_df=users,
    tracks_df=tracks,
    sessions_df=d
)

fm_model = FMModel(
    users_df=users,
    tracks_df=tracks,
    sessions_df=d,
    lightfm_model=model_h,
    interactions=interactions,
    item_features=item_features,
)

In [455]:
base_model.predict_single(123, N)

10361    7861kBrZ5aw7o4NfggcyYW
10330    5oqukcHWfqq7HGjyKwLX1y
10200    0ySCYRXVugL9xDBecGc0jr
10341    6K8qKeWo5MsFED7wCR6Kop
10220    1q8E1FfFuhd12c5JcJwPxQ
10306    4z8ssgZfs5TzKiO8HaGWXO
10352    6c8QH9FMFKj0ZgNeiyOx4v
10357    70LrxJ5u19umvrXbC19g20
10373    7qtAgn9mwxygsPOsUDVRRt
10237    2grAr8pWMuLWn8ZYEE9wDV
10353    6mAuw7R6u4vlNneuBgnC8K
10271    3nDYsXggRQxf7PCNUjR4rz
10234    2eAvDnpXP5W0cVtiI0PUxV
10289    4S9zTd8Oj77G0WhHK3Apuk
10250    34oWbFBfGEElvgO0a5c9V4
10177    02fZLz9BmmOKb5wkueM9D4
10342    6Kkt27YmFyIFrcX3QXFi2o
10369    7iPlcFvOMOzt6v0QvcAueZ
10213    1ZqHjApl3pfzwjweTfMi0g
10187    0LcNMuOiULmxJK3bdHTfDF
Name: track_id, dtype: object

In [456]:
fm_model.predict_single(123, N)

0     7qiZfU4dY1lWllzX7mPBI3
1     54bFM56PmE4YLRnqpW6Tha
2     3eekarcy7kvN4yt5ZFzltW
3     7FdUvDkaE24o3FPIWTvzv2
4     3jjujdWJ72nww5eGnfs2E7
5     2xLMifQCjDGFmkHkpNLD9h
6     7szuecWAPwGoV1e5vGu8tl
7     43zdsphuZLzwA9k4DJhU0I
8     6KfoDhO4XUWSbnyKjNp9c4
9     2b8fOow8UzyDFAE27YhOZM
10    2QjOHCTQ1Jl3zawyYOpxh6
11    35mvY5S1H3J2QZyna3TFe0
12    0nbXyq5TXYPCO7pr3N8S4I
13    2gMXnyrvIjhVBUZwvLZDMP
14    5FVd6KXrgO9B3JPmC8OPst
15    61KpQadow081I2AsbeLcsb
16    3AJwUDP919kvQ9QcozQPxg
17    2Fxmhks0bxGSBdJ92vM42m
18    086myS9r57YsLbJpU0TgK9
19    1XXimziG1uhM0eDNCZCrUl
Name: id, dtype: object

In [457]:
res_base = base_model.predict_multiple((123, 124, 125, 126, 127, 128), N)
display(res_base)

10221    1x6jHJGczTBitBy06hIWgx
11175    6LhUsJiXAuxoalUz01evh0
10598    1dWoKufYqWrjfxYhFnHyMP
11860    7FjKU2mfpEpvKIbx8naZRk
10278    42Uw9frN5ZVX40mRU3hHFL
10841    3Ud6fFep5ZlXzRWw6Sm8no
12093    52BJrldggFQyTiW8XrILhO
10387    02ppMPbg1OtEdHgoPqoqju
11864    7JElWLgpQ7lxN1JNIYwuVu
10330    5oqukcHWfqq7HGjyKwLX1y
10355    6r2BECwMgEoRb5yLfp0Hca
12558    2GrOq1y5gksYrqkc8Jzl9T
12514    1iaIEmZHrjGzWUmJ9BaFr6
12156    6vBMMa2erexusV4XrgXea9
12177    7GtGeeChOx4NS77bqK8SUx
11263    7CZiDzGVjUssMSOXrDNYHL
12465    0yJi7eb2SosK5CsSnnqc5o
11503    2Ow4Pmi0VOOLvbmJ8V70qo
10644    1xxPQaR29dAf3yxaafByeD
11199    6bwuLPr45wsvbTjB0PGI65
Name: track_id, dtype: object

In [458]:
res_fm = fm_model.predict_multiple((123, 124, 125, 126, 127, 128), N)
display(res_fm)

0     1232.666667
1     1815.166667
2     1961.000000
3     2216.666667
4     2223.500000
5     2256.500000
6     2314.000000
7     2348.000000
8     2356.333333
9     2378.166667
10    2380.333333
11    2435.666667
12    2471.000000
13    2487.000000
14    2545.333333
15    2555.500000
16    2577.833333
17    2594.000000
18    2617.166667
19    2652.166667
dtype: float64

In [460]:
dist_predicted1 = mean_dist_from_cluster_center(res_m.index, fm_model.model)
dist_sampled1 = sum([mean_dist_from_cluster_center(
    pd.concat(
    (d.loc[d['user_id'] == i] for i in (123, 124, 125, 126, 127, 128))
)['track_id'].sample(N), 
    fm_model.model) for _ in range(REPS)]) / REPS

display(pd.DataFrame(
    data={
        "method": ["Predicted by model", "Sampled from history"],
        "mean distance": [dist_predicted1, dist_sampled1]
    }
))

,method,mean distance
0,Predicted by model,1.101236
1,Sampled from history,1.451138
